In [ ]:
import pandas as pd 
import spotipy 
import spotipy.util as sp_util
from spotipy.oauth2 import SpotifyClientCredentials 
import seaborn as sb
from dotenv import load_dotenv
from os.path import join, dirname
import os

%matplotlib inline
dotenv_path = '.env'
load_dotenv(dotenv_path)

cid = os.environ.get("cid")
secret = os.environ.get("secret")
user_id= os.environ.get("user_id")

In [ ]:
sp = spotipy.Spotify() 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret,) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

In [ ]:
#spotify:user:12124178452
playlist = sp.user_playlists(user=user_id, limit=50)

#print(playlist['items'][0])
playlist_id = []

for x in playlist['items']:
    playlist_id.append(x['id'])
    info ='{} => {} total: {}'.format(x['id'], x['name'], x['tracks']['total'])
    print(info)
    

In [ ]:
# arreglo con los detalles de las canciones
features = []

#para cada id obtengo las cacnciones y los detallesa
for x in playlist_id:
    try:
        playlist = sp.user_playlist_tracks(user=user_id, playlist_id=x, limit=100)
        songs = playlist["items"]
        ids = [] 

        for i in range(len(songs)): 
            ids.append(songs[i]["track"]["id"]) 
        features.extend(sp.audio_features(ids))
        while playlist['next']:
            ids = [] 
            playlist = sp.next(playlist)
            songs = playlist["items"]
            for i in range(len(songs)): 
                ids.append(songs[i]["track"]["id"]) 
            features.extend(sp.audio_features(ids))
    except Exception:
        print('error')

# imprimo el detalle de las canciones
print(len(features))

In [ ]:
df = pd.DataFrame(features)
#https://api.spotify.com/v1/users/12124178452/playlists/3OK07Ts6uAS40k0LpOtIqW/tracks?offset=100&limit=100

df.drop_duplicates(inplace=True)
df.to_csv('audio_features.csv')
df.head()


In [ ]:
max_valence = df.valence.max()

max_valence = df[df['valence']==max_valence]

max_valence_track = sp.track(track_id=max_valence.id.iloc[0])
info = 'La más alegre: {} - {}'.format(max_valence_track['artists'][0]['name'], max_valence_track['name'])
print(info)

min_valence = df.valence.min()
min_valence_track = df[df['valence'] == min_valence]

min_valence_track = sp.track(track_id=min_valence_track.id.iloc[0])
info = 'La más triste: {} - {}'.format(min_valence_track['artists'][0]['name'], min_valence_track['name'])
print(info)

In [ ]:
sb.distplot(df.valence)

In [ ]:
sb.distplot(df.danceability)

In [ ]:
max_dance = df.danceability.max()
max_dance_track = df[df['danceability']==max_dance]

max_valence_track = sp.track(track_id=max_dance_track.id.iloc[0])
info = 'La más bailable: {} - {}'.format(max_valence_track['artists'][0]['name'], max_valence_track['name'])
print(info)


min_dance = df.danceability.min()
min_dance_track = df[df['danceability']==min_dance]

min_valence_track = sp.track(track_id=min_dance_track.id.iloc[0])
info = 'La menos bailable: {} - {}'.format(min_valence_track['artists'][0]['name'], min_valence_track['name'])
print(info)